In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import threading
import pandas as pd

Alpha = 0.1# 0.01 0.001

class Tangle(object):

    def __init__(self, env, rate=50, alpha=Alpha, tip_selection='mcmc', plot=False):
        self.time = 1.0
        self.rate = rate
        self.alpha = alpha
        
        
        if plot:
            self.G = nx.DiGraph()
        self.genesis = Genesis(self)
        self.transactions = [self.genesis]
        self.count = 0
        self.tip_selection = tip_selection

        self.cw_cache = dict()
        self.t_cache = set()
        self.tip_walk_cache = list()
        self.try_tip_walk_cache = list()

    def next_transaction(self, env):
        print(f'the time is {env.now}')
        dt_time = ( np.random.exponential(1.0/self.rate))
        wait_time = int(round(dt_time, 3)*1000)
        print(f'time needed to enter a transaction in the Tangle {wait_time}')
        self.time += dt_time 
        self.count += 1

        if self.tip_selection == 'mcmc':
            approved_tips = set(self.mcmc())
        elif self.tip_selection == 'urts':
            approved_tips = set(self.urts())
        else:
            raise Exception()

        transaction = Transaction(self, self.time, approved_tips, self.count - 1)
        print(f'the <Transaction {self.count}> is attached to {approved_tips}')
        for t in approved_tips:
            t.approved_time = np.minimum(self.time, t.approved_time)
            t._approved_directly_by.add(transaction)
            if hasattr(self, 'G'):
                self.G.add_edges_from([(transaction.num, t.num)])

        self.transactions.append(transaction)

        self.cw_cache = {}
        return dt_time
        
    def tips(self):
        return [t for t in self.transactions if t.is_visible() and t.is_tip_delayed()]

    def urts(self):
        tips = self.tips()
        if len(tips) == 0:
            return np.random.choice([t for t in self.transactions if t.is_visible()]),
        if len(tips) == 1:
            return tips[0],
        return np.random.choice(tips, 2)

    def mcmc(self):
        num_particles = 10 
        lower_bound = int(np.maximum(0, self.count - 20.0*self.rate))
        upper_bound = int(np.maximum(1, self.count - 10.0*self.rate))
        
        candidates = self.transactions[lower_bound:upper_bound]
        
        particles = np.random.choice(candidates, num_particles)
        try_particles = np.random.choice(candidates, 100)
        distances = {}
        list_confidence = list()
        if (self.count % 1000) == 0:
            for c in try_particles:
                t = threading.Thread(target=self._walk2(c))
                t.start()
                if len(self.tip_walk_cache) >=2 :
                    list_confidence += self.tip_walk_cache
                    self.tip_walk_cache = list()
            self.tip_walk_cache = list() 
            list_confidence = pd.Series(list_confidence)
            print(list_confidence.value_counts().to_dict())
            del list_confidence
        for p in particles:
            t = threading.Thread(target=self._walk2(p))
            t.start()
        tips = self.tip_walk_cache[:2]
        self.tip_walk_cache = list()

        return tips

    def _walk2(self, starting_transaction):
        p = starting_transaction
        while not p.is_tip_delayed() and p.is_visible():
            if len(self.tip_walk_cache) >= 2:
                return

            next_transactions = p.approved_directly_by()
            if self.alpha > 0:
                p_cw = p.cumulative_weight_delayed()
                c_weights = np.array([])
                for transaction in next_transactions:
                    c_weights = np.append(c_weights, transaction.cumulative_weight_delayed())

                deno = np.sum(np.exp(-self.alpha * (p_cw - c_weights)))
                probs = np.divide(np.exp(-self.alpha * (p_cw - c_weights)), deno)
            else:
                probs = None

            p = np.random.choice(next_transactions, p=probs)
            #print(p)
        self.tip_walk_cache.append(p)
        
    def _trywalk(self, starting_transaction):
        p = starting_transaction
        while not p.is_tip_delayed() and p.is_visible():
            if len(self.try_tip_walk_cache) >= 10:
                print(self.try_tip_walk_cache)
                return

            next_transactions = p.approved_directly_by()
            if self.alpha > 0:
                p_cw = p.cumulative_weight_delayed()
                c_weights = np.array([])
                for transaction in next_transactions:
                    c_weights = np.append(c_weights, transaction.cumulative_weight_delayed())

                deno = np.sum(np.exp(-self.alpha * (p_cw - c_weights)))
                probs = np.divide(np.exp(-self.alpha * (p_cw - c_weights)), deno)
            else:
                probs = None

            p = np.random.choice(next_transactions, p=probs)
        self.try_tip_walk_cache.append(p)
    

    def plot(self):
        if hasattr(self, 'G'):
            pos = nx.get_node_attributes(self.G, 'pos')
            nx.draw_networkx_nodes(self.G, pos)
            nx.draw_networkx_labels(self.G, pos)
            nx.draw_networkx_edges(self.G, pos, edgelist=self.G.edges(), arrows=True)
            plt.xlabel('Time')
            plt.yticks([])
            plt.show()


class Transaction(object):

    def __init__(self, tangle, time, approved_transactions, num):
        self.tangle = tangle
        self.time = time
        self.approved_transactions = approved_transactions
        self._approved_directly_by = set()
        self.approved_time = float('inf')
        self.num = num
        self._approved_by = set()

        if hasattr(self.tangle, 'G'):
            self.tangle.G.add_node(self.num, pos=(self.time, np.random.uniform(-1, 1)))

    def is_visible(self):
        return self.tangle.time >= self.time + 1.0

    def is_tip(self):
        return self.tangle.time < self.approved_time

    def is_tip_delayed(self):
        return self.tangle.time - 1.0 < self.approved_time

    def cumulative_weight(self):
        cw = 1 + len(self.approved_by())
        self.tangle.t_cache = set()

        return cw

    def cumulative_weight_delayed(self):
        cached = self.tangle.cw_cache.get(self.num)
        if cached:
            return cached
        else:
            cached = 1 + len(self.approved_by_delayed())
            self.tangle.t_cache = set()
            self.tangle.cw_cache[self.num] = cached

        return cached

    def approved_by(self):
        for t in self._approved_directly_by:
            if t not in self.tangle.t_cache:
                self.tangle.t_cache.add(t)
                self.tangle.t_cache.update(t.approved_by())

        return self.tangle.t_cache

    def approved_by_delayed(self):
        for t in self.approved_directly_by():
            if t not in self.tangle.t_cache:
                self.tangle.t_cache.add(t)
                self.tangle.t_cache.update(t.approved_by_delayed())

        return self.tangle.t_cache

    def approved_directly_by(self):
        return [p for p in self._approved_directly_by if p.is_visible()]

    def __repr__(self):
        return '<Transaction {}>'.format(self.num)


class Genesis(Transaction):

    def __init__(self, tangle):
        self.tangle = tangle
        self.time = 0
        self.approved_transactions = []
        self.approved_time = float('inf')
        self._approved_directly_by = set()
        self.num = 0
        if hasattr(self.tangle, 'G'):
            self.tangle.G.add_node(self.num, pos=(self.time, 0))

    def __repr__(self):
        return '<Genesis>'

In [ ]:
"""
TEST IOTA node message generator
With Author and Subscriber 
"""
import random
import numpy
import simpy
import matplotlib.pyplot as plt
import pandas as pd


RANDOM_SEED = 42 #useful only in the test phase
SIM_TIME = 500000000 #500000
node_number = 1 #10 20 40
NUM_NODES = list(range(1,node_number+1))
PTC = 0
PTC = pd.DataFrame([PTC])
rates = 10 #50 100

class Pila(object):
    """A Broadcast pipe that allows one process to send messages to many.

    This construct is useful when message consumers are running at
    different rates than message generators and provides an event
    buffering to the consuming processes.

    The parameters are used to create a new
    :class:`~simpy.resources.store.Store` instance each time
    :meth:`get_output_conn()` is called.

    """ 
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []
        

    def put(self, value, PTC):
        """Broadcast a *value* to all receivers."""
        if not self.pipes:
            raise RuntimeError('There are no output pipes.')
        events = [store.put(value) for store in self.pipes]
        PTC.iloc[0,0] +=1
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self, PTC):
        """Get a new output connection for this broadcast pipe.

        The return value is a :class:`~simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe

        
def aut_message_generator(name, env, out_pipe, PTC):
    """A process which randomly generates messages."""
    mol_val = 1000
    msg_num = 0
    wait_time = 10
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(1, 4)*mol_val + wait_time)
        msg = (env.now, 'Aut_ID')
        time_aut = env.now
        out_pipe.put(msg, PTC)
        msg_num += 1
        
def sub_message_generator(name, env, out_pipe, PTC):
    """A process which randomly generates messages."""
    mol_val = 1000
    msg_num = 0
    wait_time = 10
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(1, 4)*mol_val + wait_time)
        msg = (env.now, name)
        time_aut = env.now
        out_pipe.put(msg, PTC)
        msg_num += 1
        
    
def IOTA_nodes(name, env, in_pipe, PTC):
    """A process which consumes messages."""
    alpha = 0.1 #0.01 0.001
    counter_lecture = 0
    nodes = NUM_NODES
    list_nodes_wt = []
    elim_list = []
    plt.figure(figsize=(25, 10))
    t = Tangle(env, rate=retes, tip_selection='mcmc', plot=True) #add new
    cursor = 0 
    while cursor < 100:
        t.next_transaction(env)
        cursor = cursor + 1
    while True:
        if PTC.iloc[0,0] > 2:
            TTE = (PTC.iloc[0,0] - 2)
            random.shuffle(nodes)
            for node in nodes[:(TTE)]:
                # Get event for message pipe
                wait_time=t.next_transaction(env)
                wait_time = int(round(wait_time, 3)*1000)#
                if wait_time==0:
                    wait_time = 1
                list_nodes_wt.append([int(node),wait_time])
                msg = yield in_pipe.get()
                PTC.iloc[0,0] -= 1
                counter_lecture += 1
                time_node = env.now
                time_inv = msg[0]
                print(f'reading number {counter_lecture} of the message {msg} from the node: {node} at time {time_node}')
                nodes.remove(node)
            for couple in range(len(list_nodes_wt)):
                placeholder = list_nodes_wt[couple]
                list_nodes_wt[couple] = [placeholder[0],placeholder[1]-1]
                placeholder = list_nodes_wt[couple] 
                if placeholder[1] == 0:
                    nodes.append(placeholder[0])
            list_nodes_wt = [duo for duo in list_nodes_wt if duo[1]!=0 ]
            yield env.timeout(1)
        else:
            for couple in range(len(list_nodes_wt)):
                placeholder = list_nodes_wt[couple]
                list_nodes_wt[couple] = [placeholder[0],placeholder[1]-1]
                placeholder = list_nodes_wt[couple] 
                if placeholder[1] == 0:
                    nodes.append(placeholder[0])
            list_nodes_wt = [duo for duo in list_nodes_wt if duo[1]!=0 ]
            nodes.sort()
            yield env.timeout(1) 
